In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!pip install pyspark
!pip install findspark
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q  https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xvf spark-2.4.4-bin-hadoop2.7.tgz

     |████████████████████████████████| 215.7MB 48kB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=c48366a307fa67a4db88b9a28d1144563ff2b9640e376d44bcc466a002135afe
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 102kB 5.9MB/s 
spark-2.4.4-bin-hadoop2.7/
spark-2.4.4-bin-hadoop2.7/R/
spark-2.4.4-bin-hadoop2.7/R/lib/
spark-2.4.4-bin-hadoop2.7/R/lib/sparkr.zip
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/R.css
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/00Index.html
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/aliases.rds
spark-2.4.4-bin

In [0]:
import torch
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import findspark
from pyspark.sql.types import StructType, StructField, FloatType, StringType
import pyspark.sql.functions as F 
import sklearn
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import pyspark.mllib
import keras
import tensorflowonspark
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorframes as tfs
from pyspark.sql import Row

Using TensorFlow backend.


In [0]:
mkdir /root/.kaggle

In [0]:
cd /root/.kaggle

/root/.kaggle


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samuelpreethaml","key":"14617aaa25068d81fb4b1232097ec0ec"}'}

In [0]:
mkdir /root/recommender

In [0]:
cd /root/recommender

/root/recommender


In [0]:
!kaggle competitions download -c santander-product-recommendation

  0% 0.00/2.28M [00:00<?, ?B/s]
100% 2.28M/2.28M [00:00<00:00, 76.8MB/s]
 40% 5.00M/12.4M [00:00<00:00, 18.4MB/s]
100% 12.4M/12.4M [00:00<00:00, 35.7MB/s]
 98% 209M/214M [00:04<00:00, 46.1MB/s]
100% 214M/214M [00:04<00:00, 45.6MB/s]


In [0]:
!unzip train_ver2.csv.zip
!unzip test_ver2.csv.zip

Archive:  train_ver2.csv.zip
  inflating: train_ver2.csv          
Archive:  test_ver2.csv.zip
  inflating: test_ver2.csv           


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages databricks:tensorframes:0.6.0-s_2.11 pyspark-shell'

In [0]:
sc = SparkContext(master='local',appName='test1')

In [0]:
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [0]:
df = spark.read.csv(path='train_ver2.csv',sep=',',header=True,inferSchema=True)

In [0]:
df.printSchema()

root
 |-- fecha_dato: timestamp (nullable = true)
 |-- ncodpers: double (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: string (nullable = true)
 |-- fecha_alta: timestamp (nullable = true)
 |-- ind_nuevo: string (nullable = true)
 |-- antiguedad: string (nullable = true)
 |-- indrel: string (nullable = true)
 |-- ult_fec_cli_1t: timestamp (nullable = true)
 |-- indrel_1mes: string (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: string (nullable = true)
 |-- indext: string (nullable = true)
 |-- conyuemp: string (nullable = true)
 |-- canal_entrada: string (nullable = true)
 |-- indfall: string (nullable = true)
 |-- tipodom: string (nullable = true)
 |-- cod_prov: string (nullable = true)
 |-- nomprov: string (nullable = true)
 |-- ind_actividad_cliente: string (nullable = true)
 |-- renta: double (nullable = true)
 |-- segmento: string (nullable = true

In [0]:
drop_list = ['ind_empleado','pais_residencia','fecha_alta','indrel','ult_fec_cli_1t','indrel_1mes','indresi','conyuemp','indfall','tipodom','nomprov']
dfa = df.select([column for column in df.columns if column not in drop_list])

In [0]:
dfa=dfa.fillna({'renta':1})
dfa=dfa.withColumn("antiguedad",F.when(dfa["antiguedad"] =='     NA',0).otherwise(dfa["antiguedad"]))
dfa=dfa.withColumn("antiguedad",F.when(dfa["antiguedad"] =='-999999',0).otherwise(dfa["antiguedad"]))
dfa=dfa.withColumn("antiguedad",F.when(F.isnull('antiguedad'),0).otherwise(dfa["antiguedad"]))
dfa=dfa.withColumn("sexo",F.when(F.isnull('sexo'),'V').otherwise(dfa["sexo"]))
dfa=dfa.withColumn("age",F.when(F.isnull('age'),25).otherwise(dfa["age"]))
dfa=dfa.withColumn("age",F.when(dfa["age"] ==' NA',25).otherwise(dfa["age"]))
dfa=dfa.withColumn("tiprel_1mes",F.when(F.isnull('tiprel_1mes'),'A').otherwise(dfa["tiprel_1mes"]))
dfa=dfa.withColumn("indext",F.when(F.isnull('indext'),'N').otherwise(dfa["indext"]))
dfa=dfa.withColumn("canal_entrada",F.when(F.isnull('canal_entrada'),'KHE').otherwise(dfa["canal_entrada"]))
dfa=dfa.withColumn("segmento",F.when(F.isnull('segmento'),'02 - PARTICULARES').otherwise(dfa["segmento"]))
dfa=dfa.withColumn("cod_prov",F.when(F.isnull('cod_prov'),50).otherwise(dfa["cod_prov"]))
dfa=dfa.withColumn("ind_nuevo",F.when(F.isnull('ind_nuevo'),0).otherwise(dfa["ind_nuevo"]))
dfa=dfa.withColumn("ind_actividad_cliente",F.when(F.isnull('ind_actividad_cliente'),0).otherwise(dfa["ind_actividad_cliente"]))

In [0]:
dfb = dfa.withColumn('total', sum(dfa[col] for col in df.columns[-24:]))

In [0]:
dfc=dfb.filter(dfb.total!=0)

In [0]:
dfd=dfc.fillna(0)

In [0]:
dfe=dfd.withColumn("fecha_year",F.year(F.col("fecha_dato")))
dff=dfe.withColumn("fecha_month",F.month(F.col("fecha_dato")))
dfg=dff.withColumn("fecha_year",F.when(dff["fecha_year"] ==2015,0).otherwise(1))

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dfg) for column in ['sexo','tiprel_1mes','indext','canal_entrada','segmento']]


pipeline = Pipeline(stages=indexers)
dfl = pipeline.fit(dfg).transform(dfg)
dfl.show()

+-------------------+---------+----+---+---------+----------+-----------+------+-------------+--------+---------------------+------------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+-----+----------+-----------+----------+-----------------+------------+-------------------+--------------+
|         fecha_dato| ncodpers|sexo|age|ind_nuevo|antiguedad|tiprel_1mes|indext|canal_entrada|cod_prov|ind_actividad_cliente|             renta|          segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|i

In [0]:
drop_list = ['fecha_dato','sexo','tiprel_1mes','indext','canal_entrada','segmento']
dfk = dfl.select([column for column in dfl.columns if column not in drop_list])
dfk=dfk.withColumn("ind_nuevo",F.when(F.isnull('ind_nuevo'),0).otherwise(dfk["ind_nuevo"]))
dfk=dfk.withColumn("cod_prov",F.when(F.isnull('cod_prov'),50).otherwise(dfk["cod_prov"]))
dfk=dfk.withColumn("ind_actividad_cliente",F.when(F.isnull('ind_actividad_cliente'),0).otherwise(dfk["ind_actividad_cliente"]))
dfk=dfk.withColumn("ind_nuevo",F.when(dfk["ind_nuevo"]=='NA',0).otherwise(dfk["ind_nuevo"]))
dfk=dfk.withColumn("cod_prov",F.when(dfk["cod_prov"]=='NA',50).otherwise(dfk["cod_prov"]))
dfk=dfk.withColumn("ind_actividad_cliente",F.when(dfk["ind_actividad_cliente"]=='NA',50).otherwise(dfk["ind_actividad_cliente"]))

In [0]:
data=dfk.select(dfk.ncodpers.cast('double'),dfk.age.cast('float'),dfk.ind_nuevo.cast('float'),dfk.antiguedad.cast('float'),dfk.cod_prov.cast('float'),dfk.ind_actividad_cliente.cast('float'),dfk.renta.cast('double'),dfk.sexo_index.cast('float'),dfk.tiprel_1mes_index.cast('float'),dfk.indext_index.cast('float'),dfk.canal_entrada_index.cast('float'),dfk.segmento_index.cast('float'),dfk.fecha_year.cast('integer'),dfk.fecha_month.cast('integer'),dfk.total.cast('integer'),dfk.ind_ahor_fin_ult1.cast('integer'),dfk.ind_aval_fin_ult1.cast('integer'),dfk.ind_cco_fin_ult1.cast('integer'),dfk.ind_cder_fin_ult1.cast('integer'),dfk.ind_cno_fin_ult1.cast('integer'),dfk.ind_ctju_fin_ult1.cast('integer'),dfk.ind_ctma_fin_ult1.cast('integer'),dfk.ind_ctop_fin_ult1.cast('integer'),dfk.ind_ctpp_fin_ult1.cast('integer'),dfk.ind_deco_fin_ult1.cast('integer'),dfk.ind_deme_fin_ult1.cast('integer'),dfk.ind_dela_fin_ult1.cast('integer'),dfk.ind_ecue_fin_ult1.cast('integer'),dfk.ind_fond_fin_ult1.cast('integer'),dfk.ind_hip_fin_ult1.cast('integer'),dfk.ind_plan_fin_ult1.cast('integer'),dfk.ind_pres_fin_ult1.cast('integer'),dfk.ind_reca_fin_ult1.cast('integer'),dfk.ind_tjcr_fin_ult1.cast('integer'),dfk.ind_valo_fin_ult1.cast('integer'),dfk.ind_viv_fin_ult1.cast('integer'),dfk.ind_nomina_ult1.cast('float'),dfk.ind_nom_pens_ult1.cast('float'),dfk.ind_recibo_ult1.cast('integer'))


In [0]:
data.show(10)

+---------+----+---------+----------+--------+---------------------+------------------+----------+-----------------+------------+-------------------+--------------+----------+-----------+-----+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
| ncodpers| age|ind_nuevo|antiguedad|cod_prov|ind_actividad_cliente|             renta|sexo_index|tiprel_1mes_index|indext_index|canal_entrada_index|segmento_index|fecha_year|fecha_month|total|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|ind_ctpp_fin_ult1|ind_deco_fin_ult1|ind_deme_fi

In [0]:
vecAssembler = VectorAssembler(inputCols=["ncodpers","age","ind_nuevo","antiguedad","cod_prov","ind_actividad_cliente","renta","sexo_index","tiprel_1mes_index","indext_index","canal_entrada_index","segmento_index","fecha_year","fecha_month","total"], outputCol='features')
dt=vecAssembler.transform(data)

In [0]:
vecAssembler = VectorAssembler(inputCols=["ind_ahor_fin_ult1","ind_aval_fin_ult1","ind_cco_fin_ult1",'ind_cder_fin_ult1', 'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1', 'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'], outputCol='label')
dts=vecAssembler.transform(dt)

In [0]:
dts.show(10)

+---------+----+---------+----------+--------+---------------------+------------------+----------+-----------------+------------+-------------------+--------------+----------+-----------+-----+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+--------------------+--------------+
| ncodpers| age|ind_nuevo|antiguedad|cod_prov|ind_actividad_cliente|             renta|sexo_index|tiprel_1mes_index|indext_index|canal_entrada_index|segmento_index|fecha_year|fecha_month|total|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|ind_ctpp_fi

In [0]:
ncodpers=tfs.row(data,'ncodpers')
ncod=ncodpers

In [0]:
tfs.map_rows(ncod,data)

Py4JJavaError: ignored

In [0]:
df=sqlContext.createDataFrame(zip(range(10),range(1,11))).toDF('x','y')
df.show()

+---+---+
|  x|  y|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
|  5|  6|
|  6|  7|
|  7|  8|
|  8|  9|
|  9| 10|
+---+---+



In [0]:
dir(tfs)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'absolute_import',
 'aggregate',
 'analyze',
 'block',
 'core',
 'map_blocks',
 'map_rows',
 'print_schema',
 'reduce_blocks',
 'reduce_rows',
 'row']

In [0]:
from pyspark.ml.classification import LogisticRegression
for col in train.columns[15:39]:
  lr = LogisticRegression(labelCol=col, featuresCol="features",maxIter=10)
  model = lr.fit(train)
  model.save('/root/pyspark/'+col)

In [0]:
predictions=lrModel.transform(test)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol='ind_ahor_fin_ult1')
evaluator.evaluate(predictions)

0.5

In [0]:
from pyspark.mllib.recommendation import ALS, Rating
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

# Read in the ratings data
lines = sc.textFile("data/mllib/sample_movielens_data.txt")

def parseLine(line):
    fields = line.split("::")
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]) - 2.5)
ratings = lines.map(lambda r: parseLine(r))

# Train a model on to predict user-product ratings
model = ALS.train(ratings, 10, 10, 0.01)

# Get predicted ratings on all existing user-product pairs
testData = ratings.map(lambda p: (p.user, p.product))
predictions = model.predictAll(testData).map(lambda r: ((r.user, r.product), r.rating))

ratingsTuple = ratings.map(lambda r: ((r.user, r.product), r.rating))
scoreAndLabels = predictions.join(ratingsTuple).map(lambda tup: tup[1])

# Instantiate regression metrics to compare predicted and actual ratings
metrics = RegressionMetrics(scoreAndLabels)

# Root mean squared error
print("RMSE = %s" % metrics.rootMeanSquaredError)

# R-squared
print("R-squared = %s" % metrics.r2)

AttributeError: ignored

In [0]:
!cd /root/pyspark

In [0]:
r1 = (1, (1.0,2.0),1)
r2 = (1, (2.0,1.0),2)
r3 = (2, (2.0,2.0),2)
ratings = sc.parallelize([r1, r2, r3])
model = SVMWithSGD.train(ratings, iterations=100, step=1.0, regParam=0.01, miniBatchFraction=1.0, initialWeights=None, regType='l2', intercept=False, validateData=True, convergenceTol=0.001)
model.predict(2, 2)

NameError: ignored

In [0]:
predictions.show(10)

+--------+----+---------+----------+--------+---------------------+-----------------+----------+-----------------+------------+-------------------+--------------+----------+-----------+-----+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------------+----------+
|ncodpers| age|ind_nuevo|antiguedad|cod_prov|ind_actividad_cliente|            renta|sexo_index|tiprel_1mes_index|indext_index|canal_entrada_index|segmento_index|fecha_year|fecha_month|total|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_f

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol='ind_ahor_fin_ult1')
evaluator.evaluate(predictions)

0.8861058773121893

In [0]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
dataA = [(0, Vectors.dense([1.0, 1.0]),),
         (1, Vectors.dense([1.0, -1.0]),),
         (2, Vectors.dense([-1.0, -1.0]),),
         (3, Vectors.dense([-1.0, 1.0]),)]
dfA = spark.createDataFrame(dataA, ["id", "features"])

dataB = [(Vectors.dense([1.0, 0.0]), Vectors.dense([1.0, 0.0]),),
         (Vectors.dense([1.0, 0.0]), Vectors.dense([-1.0, 0.0]),),
         (Vectors.dense([1.0, 0.0]), Vectors.dense([0.0, 1.0]),),
         (Vectors.dense([1.0, 0.0]), Vectors.dense([0.0, -1.0]),)]
dfB = spark.createDataFrame(dataB, ["id", "features"])

key = Vectors.dense([1.0, 0.0,2])

brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,
                                  numHashTables=3)
model = brp.fit(dfB)

# Feature Transformation
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(dfA).show()

The hashed dataset where hashed values are stored in the column 'hashes':
+---+-----------+--------------------+
| id|   features|              hashes|
+---+-----------+--------------------+
|  0|  [1.0,1.0]|[[-1.0], [0.0], [...|
|  1| [1.0,-1.0]|[[-1.0], [0.0], [...|
|  2|[-1.0,-1.0]|[[0.0], [-1.0], [...|
|  3| [-1.0,1.0]|[[0.0], [-1.0], [...|
+---+-----------+--------------------+



In [0]:
sc1.uiWebUrl

<!DOCTYPE html><html>
      <head>
        <meta http-equiv="Content-type" content="text/html; charset=utf-8"/><link rel="stylesheet" href="/static/bootstrap.min.css" type="text/css"/><link rel="stylesheet" href="/static/vis.min.css" type="text/css"/><link rel="stylesheet" href="/static/webui.css" type="text/css"/><link rel="stylesheet" href="/static/timeline-view.css" type="text/css"/><script src="/static/sorttable.js"></script><script src="/static/jquery-1.12.4.min.js"></script><script src="/static/vis.min.js"></script><script src="/static/bootstrap-tooltip.js"></script><script src="/static/initialize-tooltips.js"></script><script src="/static/table.js"></script><script src="/static/additional-metrics.js"></script><script src="/static/timeline-view.js"></script><script src="/static/log-view.js"></script><script src="/static/webui.js"></script><script>setUIRoot('')</script>
        
        
        <link rel="shortcut icon" href="/static/spark-logo-77x50px-hd.png"></link>
        <ti

In [0]:
import xgboost as xgb
import pandas as pd
import numpy as np
import gzip
import math
import pickle
import zlib
import io
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from matplotlib import pyplot as pt
import tensorboard
import tensorflow
from torch.utils.data import DataLoader, Dataset
import torchvision
from collections import defaultdict
from tqdm import tqdm, tqdm_notebook, tnrange

In [0]:
train, test=data.randomSplit([0.7, 0.3], seed = 123)
train.show(10)

+--------+----+---------+----------+--------+---------------------+------------------+----------+-----------------+------------+-------------------+--------------+----------+-----------+-----+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|ncodpers| age|ind_nuevo|antiguedad|cod_prov|ind_actividad_cliente|             renta|sexo_index|tiprel_1mes_index|indext_index|canal_entrada_index|segmento_index|fecha_year|fecha_month|total|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|ind_ctpp_fin_ult1|ind_deco_fin_ult1|ind_deme_fin_

In [0]:
class RegressionColumnarDataset(Dataset):
    def __init__(self, df, cats,dy):
        self.dfcats = df[cats]
        self.dfconts = df.drop(cats, axis=1)
        self.dy=dy

        
        self.cats = np.stack([c.values for n, c in self.dfcats.items()], axis=1).astype(np.int64)
        self.conts = np.stack([c.values for n, c in self.dfconts.items()], axis=1).astype(np.float32)
        self.y = np.stack([c.values for n, c in self.dy.items()], axis=1).astype(np.float32)
        
    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]

In [0]:
import sparkdl

AttributeError: ignored

In [0]:
train.items()

AttributeError: ignored